In [14]:
import glob
import re

import numpy as np
import pandas as pd
import tqdm
import itertools
from collections import defaultdict
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')
# pd.options.display.max_columns = 2000

<ipython-input-14-d205f42a1420>:12: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg')


In [25]:
class IteratorLogs():

    def __init__(self, logfile) -> None:
        with open(logfile) as f:
            self.lines = f.readlines()
        self.index = 0

    def next(self):
        temp = self.next_line()
        # filter GLOG
        while temp.find('.h:') != -1 or temp.find('.cc:') != -1:
            temp = self.next_line()
        # print(self.index, temp)
        return temp

    def next_line(self):
        if self.index == len(self.lines):
            raise StopIteration()
        temp = self.lines[self.index]
        self.index += 1
        return temp


# run_log_file to [{ts: ts1, k1: v, k2: v}, (ts2, {})]
def parse_log(logfile, isclient=False):
    iter_log = IteratorLogs(logfile)

    dic = defaultdict(lambda: [])
    i = 0
    while True:
        try:
            each_line = iter_log.next()
            each_line = each_line.strip()
            r = re.search(r'\[proc (\d)\] (\d+) steps, (total:.*$)', each_line)
            if r:
                proc = int(r[1])
                step = int(r[2])
                items = r[3].split(", ")
                items = [(each.split(": ")[0], each.split(": ")[1]) for each in items]
                for each_k, each_v in items:
                    dic[f'proc{proc}_{each_k}'].append(float(each_v))
                continue
            
        except StopIteration:
            # if 'ts' in dic.keys():
            #     results.append(dic)
            break
    
    for each_k, each_v in list(dic.items()):
        dic[f"{each_k}_jicha"] = max(each_v[-3:]) - min(each_v[-3:])
        dic[each_k] = np.mean(each_v[-3:])

    return dic

In [26]:
parse_log("benchmark/log/exp0-single-3090/run_0/log")

defaultdict(<function __main__.parse_log.<locals>.<lambda>()>,
            {'proc0_total': 36.25633333333334,
             'proc0_sample': 1.529,
             'proc0_forward': 7.997,
             'proc0_backward': 23.468333333333334,
             'proc0_update': 3.254,
             'proc0_total_jicha': 0.39800000000000324,
             'proc0_sample_jicha': 0.10899999999999999,
             'proc0_forward_jicha': 0.09700000000000042,
             'proc0_backward_jicha': 0.4059999999999988,
             'proc0_update_jicha': 0.07899999999999974})